## Do zrobienia:

- Usunięcie zbędnych kolumn - takich jak np. informacje o dacie ceremonii otwarcia poszczególnych igrzysk.
- Utworzenie kolumny sport type - wiąże się to z zaklasyfikowaniem poszczególnych sportów do odpowiednio zdefiniowanych typów (np. sporty drużynowe, wodne itp.)
- Utworzenie indeksów zastępczych przy tabelach sportowców i igrzysk (igrzyska mogą być identyfikowane przez połączenie kolum year + season, warto więc wprowadzić numeryczny indeks, który ułatwi odwoływanie się z tabeli faktowej).
- Ujednolicenie sposobu zapisu nazw drużyn. -> Nie rozumiem.
- Ujednolicenie nazw lokalizacji (część rekordów opisuje już nieistniejące kraje, jak imperialna Japonia czy nazistowskie Niemcy.
- Dopasowanie NOC do państw (niektóre kody mogą być przestarzałe). -> Nie widzę problemu. Widocznie w roku igrzysk dane państwo nazywało się inaczej niż teraz.

In [636]:
import numpy as np
import pandas as pd
import copy
from datetime import date

In [637]:
df_athlete_events = pd.read_csv('data/new_athlete_events.csv', delimiter=';')
df_athlete_events.head(3)

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN


In [638]:
df_player_scores = pd.read_csv('data/player_scores.csv', delimiter=',')
df_player_scores.head(3)

,player,score
0,1,4
1,2,4
2,3,5


In [639]:
df_noc_regions = pd.read_csv('data/noc_regions.csv', delimiter=',')
df_noc_regions.head(3)

,NOC,region,notes
0,AFG,Afghanistan,NaN
1,AHO,Curacao,Netherlands Antilles
2,ALB,Albania,NaN


In [640]:
# w tabelce df_noc_regions nie ma singapuru

In [641]:
new_region = {'NOC':'SGP','region':'Singapore','notes':'Singapore'}

In [642]:
df_noc_regions = df_noc_regions.append(new_region,ignore_index=True)

In [643]:
len(df_noc_regions['NOC'].unique())

231

In [644]:
df_cities = pd.read_csv('df_cities_with_id_column.csv', delimiter=',')
df_cities

,location_id,City,Country,Continent,Summer (Olympiad),Winter,Year,Opening Ceremony,Closing Ceremony
0,0,Athens,Greece,Europe,I,-,1896.0,April 6,April 15
1,1,Paris,France,Europe,II,-,1900.0,May 14,October 28
2,2,St. Louis,United States,North America,III,-,1904.0,July 1,November 23
3,3,London,United Kingdom,Europe,IV,-,1908.0,April 27,October 31
4,4,Stockholm,Sweden,Europe,V,-,1912.0,May 5,July 22
5,5,Berlin,Germany,Europe,VI,-,1916.0,Cancelled due to World War I,NaN
6,6,Antwerp,Belgium,Europe,VII,-,1920.0,April 20,September 12
7,7,Chamonix,France,Europe,-,I,1924.0,January 25,February 4
8,8,Paris,France,Europe,VIII,-,NaN,May 4,July 27
9,9,St. Moritz,Switzerland,Europe,-,II,1928.0,February 11,February 19


In [645]:
df_games_data = pd.read_csv('data/games_data.csv', delimiter=',')
df_games_data.head(3)

,games,costs,athletes,events
0,1896 Summer,NaN,241,43
1,1900 Summer,NaN,997,95
2,1904 Summer,NaN,651,95


- Filtrowanie - wybranie tylko potrzebnych do hurtowni atrybutów oraz ograniczenie obserwacji do lat 2000-2016.

In [646]:
df_athlete_events = df_athlete_events[(df_athlete_events.Year >= 2000) & (df_athlete_events.Year <= 2015)]
df_athlete_events.reset_index(drop=True, inplace=True)

In [647]:
df_cities = df_cities[(df_cities.Year >= 2000) & (df_cities.Year <= 2015)]
df_cities.reset_index(drop=True, inplace=True)

In [648]:
df_cities

,location_id,City,Country,Continent,Summer (Olympiad),Winter,Year,Opening Ceremony,Closing Ceremony
0,46,Sydney,Australia,Oceania,XXVII,-,2000.0,September 15,October 1
1,47,Salt Lake City,United States,North America,-,XIX,2002.0,February 8,February 24
2,48,Athens,Greece,Europe,XXVIII,-,2004.0,August 13,August 29
3,49,Turin,Italy,Europe,-,XX,2006.0,February 10,February 26
4,50,Beijing,China,Asia,XXIX,-,2008.0,August 8,August 24
5,51,Vancouver,Canada,North America,-,XXI,2010.0,February 12,February 28
6,52,London,United Kingdom,Europe,XXX,-,2012.0,July 27,August 12
7,53,Sochi,Russia,Europe,-,XXII,2014.0,February 7,February 23


## Utworzenie tabelki `Locations`

In [649]:
Locations = df_cities.drop_duplicates(subset=['Continent', 'Country', 'City'])[['Continent', 'Country', 'City']]
Locations.columns = ['continent', 'country', 'city']
Locations.insert(loc=0, column='location_id', value=range(len(Locations)))
Locations

,location_id,continent,country,city
0,0,Oceania,Australia,Sydney
1,1,North America,United States,Salt Lake City
2,2,Europe,Greece,Athens
3,3,Europe,Italy,Turin
4,4,Asia,China,Beijing
5,5,North America,Canada,Vancouver
6,6,Europe,United Kingdom,London
7,7,Europe,Russia,Sochi


## Utworzenie tabelki `Teams`
- Zamiana pustych wartości parametru team country na nazwę kraju.

In [650]:
Teams = df_athlete_events.drop_duplicates(subset=['Team', 'NOC'])[['Team', 'Team', 'NOC']]
Teams.reset_index(inplace=True, drop=True)
Teams.columns = ['team_name', 'team_country', 'NOC']
Teams.insert(loc=0, column='team_id', value=range(len(Teams)))
Teams

,team_id,team_name,team_country,NOC
0,0,China,China,CHN
1,1,Finland,Finland,FIN
2,2,Norway,Norway,NOR
3,3,Netherlands,Netherlands,NED
4,4,France,France,FRA
...,...,...,...,...
323,323,Salinero,Salinero,NED
324,324,Satchmo,Satchmo,GER
325,325,Solos Carex,Solos Carex,SWE
326,326,Dow Jones,Dow Jones,JPN


In [651]:
len(Teams["NOC"].unique())

207

- Czyszczenie - wypełnienie pustych elementów w tabeli zawodnika wartością domyślną.

In [652]:
df_athlete_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71570 entries, 0 to 71569
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      71570 non-null  int64  
 1   Name    71570 non-null  object 
 2   Sex     71570 non-null  object 
 3   Age     71567 non-null  float64
 4   Height  71035 non-null  float64
 5   Weight  70583 non-null  float64
 6   Team    71570 non-null  object 
 7   NOC     71570 non-null  object 
 8   Games   71570 non-null  object 
 9   Year    71570 non-null  int64  
 10  Season  71570 non-null  object 
 11  City    71570 non-null  object 
 12  Sport   71570 non-null  object 
 13  Event   71570 non-null  object 
 14  Medal   10115 non-null  object 
dtypes: float64(3), int64(2), object(10)
memory usage: 8.2+ MB


Puste wartości w kolumnach: `Age`, `Height`, `Weight`, `Medal`.

In [653]:
mean_age = int(np.mean(df_athlete_events.Age))
mean_height = int(np.mean(df_athlete_events.Height))
mean_weight = int(np.mean(df_athlete_events.Weight))

In [654]:
df_athlete_events.Age.fillna(mean_age, inplace=True)
df_athlete_events.Height.fillna(mean_height, inplace=True)
df_athlete_events.Weight.fillna(mean_weight, inplace=True)

- Zamienić wartości NULL w uzyskanych medalach na informację o braku medalu.

In [655]:
df_athlete_events.Medal.fillna('lack', inplace=True)

In [656]:
df_athlete_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71570 entries, 0 to 71569
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      71570 non-null  int64  
 1   Name    71570 non-null  object 
 2   Sex     71570 non-null  object 
 3   Age     71570 non-null  float64
 4   Height  71570 non-null  float64
 5   Weight  71570 non-null  float64
 6   Team    71570 non-null  object 
 7   NOC     71570 non-null  object 
 8   Games   71570 non-null  object 
 9   Year    71570 non-null  int64  
 10  Season  71570 non-null  object 
 11  City    71570 non-null  object 
 12  Sport   71570 non-null  object 
 13  Event   71570 non-null  object 
 14  Medal   71570 non-null  object 
dtypes: float64(3), int64(2), object(10)
memory usage: 8.2+ MB


Brak pustych wartości.

- Wyliczenie roku urodzenia zawodnika na podstawie wieku zawodnika i roku odbywania się igrzysk.


In [657]:
df_athlete_events['birth_year'] = df_athlete_events.Year-df_athlete_events.Age

## Utworzenie tabelki `Athletes`
- utworzenie indeksów zastępczych

In [658]:
Athletes = df_athlete_events.drop_duplicates(subset=['ID'])[['ID','Name','Sex','birth_year','Height','Weight']]
Athletes.columns = ['athlete_id','full_name','sex','birth_year','height','weight']
Athletes = Athletes.merge(df_player_scores, how='left', left_on='athlete_id', right_on='player')
del Athletes['player']

Athletes.insert(loc=0, column='sid', value=range(len(Athletes)))
Athletes.insert(loc=8, column='date_valid_from', value=date.today())
Athletes.insert(loc=9, column='date_valid_to', value=date.max)
Athletes.insert(loc=10, column='status', value=1)

Athletes.head(3)

,sid,athlete_id,full_name,sex,birth_year,height,weight,score,date_valid_from,date_valid_to,status
0,0,2,A Lamusi,M,1989.0,170.0,60.0,4,2022-06-10,9999-12-31,1
1,1,9,Antti Sami Aalto,M,1976.0,186.0,96.0,4,2022-06-10,9999-12-31,1
2,2,12,Jyri Tapani Aalto,M,1969.0,172.0,70.0,1,2022-06-10,9999-12-31,1


- Utworzenie kolumny medal score i przypisanie poszczególnym medalom odpowiedniej punktacji (opisanej w sekcji Opis kluczowych miar i atrybutów w modelu.

In [659]:
medal_val_dict = {'Gold':5, 'Silver':3, 'Bronze':1, 'lack':0}

In [660]:
df_athlete_events['medal_score'] = df_athlete_events.Medal.replace(medal_val_dict)
df_athlete_events.head(5)

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,birth_year,medal_score
0,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,lack,1989.0,0
1,9,Antti Sami Aalto,M,26.0,186.0,96.0,Finland,FIN,2002 Winter,2002,Winter,Salt Lake City,Ice Hockey,Ice Hockey Men's Ice Hockey,lack,1976.0,0
2,12,Jyri Tapani Aalto,M,31.0,172.0,70.0,Finland,FIN,2000 Summer,2000,Summer,Sydney,Badminton,Badminton Men's Singles,lack,1969.0,0
3,13,Minna Maarit Aalto,F,34.0,159.0,55.5,Finland,FIN,2000 Summer,2000,Summer,Sydney,Sailing,Sailing Women's Windsurfer,lack,1966.0,0
4,16,Juhamatti Tapio Aaltonen,M,28.0,184.0,85.0,Finland,FIN,2014 Winter,2014,Winter,Sochi,Ice Hockey,Ice Hockey Men's Ice Hockey,Bronze,1986.0,1


In [661]:
Teams.index

RangeIndex(start=0, stop=328, step=1)

In [662]:
df_athlete_events.join(Teams.set_index('team_name'), on='Team', lsuffix='_left', rsuffix='_right', how='inner')[['team_id','team_country','Team','NOC_right']].drop_duplicates()

,team_id,team_country,Team,NOC_right
0,0,China,China,CHN
1,1,Finland,Finland,FIN
6,2,Norway,Norway,NOR
16,3,Netherlands,Netherlands,NED
17,4,France,France,FRA
...,...,...,...,...
65260,323,Salinero,Salinero,NED
67736,324,Satchmo,Satchmo,GER
68050,325,Solos Carex,Solos Carex,SWE
69028,326,Dow Jones,Dow Jones,JPN


- Pominięcie danych o igrzyskach, które nie odbyły się.

In [663]:
df_cities = df_cities[df_cities['Opening Ceremony'].str.contains('Cancelled')==False]
df_cities

,location_id,City,Country,Continent,Summer (Olympiad),Winter,Year,Opening Ceremony,Closing Ceremony
0,46,Sydney,Australia,Oceania,XXVII,-,2000.0,September 15,October 1
1,47,Salt Lake City,United States,North America,-,XIX,2002.0,February 8,February 24
2,48,Athens,Greece,Europe,XXVIII,-,2004.0,August 13,August 29
3,49,Turin,Italy,Europe,-,XX,2006.0,February 10,February 26
4,50,Beijing,China,Asia,XXIX,-,2008.0,August 8,August 24
5,51,Vancouver,Canada,North America,-,XXI,2010.0,February 12,February 28
6,52,London,United Kingdom,Europe,XXX,-,2012.0,July 27,August 12
7,53,Sochi,Russia,Europe,-,XXII,2014.0,February 7,February 23


## Utworzenie tabelki `Sports`

In [664]:
sports_types_dict = {
    'team_sport' : ['Baseball', 'Basketball', 'Beach Volleyball',
         'Bobsleigh', 'Cross Country Skiing',
         'Curling', 'Football', 'Handball', 'Hockey', 'Ice Hockey', 'Nordic Combined', 
         'Rowing', 'Sailing', 'Shooting', 'Short Track Speed Skating',
         'Skeleton', 'Ski Jumping', 'Snowboarding', 'Softball',
         'Speed Skating', 'Swimming', 'Synchronized Swimming',
         'Table Tennis', 'Taekwondo', 'Tennis', 'Trampolining',
         'Triathlon', 'Volleyball', 'Water Polo', 'Weightlifting',
         'Wrestling']
}

In [665]:
individual = [a for a in np.unique(df_athlete_events.Sport) if a not in sports_types_dict['team_sport']]

In [666]:
sports_types_dict = {
    'team_sport' : ['Baseball', 'Basketball', 'Beach Volleyball',
         'Bobsleigh', 'Cross Country Skiing',
         'Curling', 'Football', 'Handball', 'Hockey', 'Ice Hockey', 'Nordic Combined', 
         'Rowing', 'Sailing', 'Shooting', 'Short Track Speed Skating',
         'Skeleton', 'Ski Jumping', 'Snowboarding', 'Softball',
         'Speed Skating', 'Swimming', 'Synchronized Swimming',
         'Table Tennis', 'Taekwondo', 'Tennis', 'Trampolining',
         'Triathlon', 'Volleyball', 'Water Polo', 'Weightlifting',
         'Wrestling'],
    'individual_sport': individual
}

Podział na sporty drużynowe/indywidualne.

In [667]:
len(sports_types_dict['team_sport'])

31

Nieskończone. Nie znam się. Może lepszy będzie podział: drużynowe/ w parach/indywidualne. Niektóre sporty mają podkategorie, w których są i dyscypliny drużynowe i indywidualne, np. Gymnastic.

In [668]:
Sports = df_athlete_events.drop_duplicates(subset=['Sport','Event'])[['Sport','Event']]
Sports.reset_index(inplace=True, drop=True)
Sports.columns = ['sport_name', 'event_name']
Sports.insert(loc=0, column='event_id', value=range(len(Sports)))

Sports

,event_id,sport_name,event_name
0,0,Judo,Judo Men's Extra-Lightweight
1,1,Ice Hockey,Ice Hockey Men's Ice Hockey
2,2,Badminton,Badminton Men's Singles
3,3,Sailing,Sailing Women's Windsurfer
4,4,Athletics,Athletics Men's Shot Put
...,...,...,...
430,430,Trampolining,Trampolining Men's Individual
431,431,Weightlifting,Weightlifting Women's Flyweight
432,432,Cycling,Cycling Women's Omnium
433,433,Cycling,Cycling Women's Team Sprint


In [669]:
Sports

,event_id,sport_name,event_name
0,0,Judo,Judo Men's Extra-Lightweight
1,1,Ice Hockey,Ice Hockey Men's Ice Hockey
2,2,Badminton,Badminton Men's Singles
3,3,Sailing,Sailing Women's Windsurfer
4,4,Athletics,Athletics Men's Shot Put
...,...,...,...
430,430,Trampolining,Trampolining Men's Individual
431,431,Weightlifting,Weightlifting Women's Flyweight
432,432,Cycling,Cycling Women's Omnium
433,433,Cycling,Cycling Women's Team Sprint


In [670]:
len(np.unique(Sports.event_name)) #unikalne

435

- Połączenie przechowywanych oddzielnie dnia i miesiąca oraz roku odbycia się ceremonii otwarcia igrzysk w jedną datę w dla wymiaru daty.

In [671]:
import datetime

In [672]:
df_cities = df_cities.astype({'Year':int})

In [673]:
import warnings
warnings.filterwarnings('ignore')

In [674]:
for i in range(len(df_cities)):
    df_cities.Year[i] = int(df_cities.Year[i])
    df_cities['Opening Ceremony'][i] = datetime.datetime(
        df_cities.Year[i],
        int(datetime.datetime.strptime(df_cities['Opening Ceremony'][i] , '%B %d').strftime('%m')),
        int(datetime.datetime.strptime(df_cities['Opening Ceremony'][i] , '%B %d').strftime('%d'))
    ).strftime('%Y-%m-%d')
df_cities

,location_id,City,Country,Continent,Summer (Olympiad),Winter,Year,Opening Ceremony,Closing Ceremony
0,46,Sydney,Australia,Oceania,XXVII,-,2000,2000-09-15,October 1
1,47,Salt Lake City,United States,North America,-,XIX,2002,2002-02-08,February 24
2,48,Athens,Greece,Europe,XXVIII,-,2004,2004-08-13,August 29
3,49,Turin,Italy,Europe,-,XX,2006,2006-02-10,February 26
4,50,Beijing,China,Asia,XXIX,-,2008,2008-08-08,August 24
5,51,Vancouver,Canada,North America,-,XXI,2010,2010-02-12,February 28
6,52,London,United Kingdom,Europe,XXX,-,2012,2012-07-27,August 12
7,53,Sochi,Russia,Europe,-,XXII,2014,2014-02-07,February 23


## Utworzenie tabelki `Dates`

In [675]:
Dates = df_cities[['Opening Ceremony','Winter','Year']]
Dates['season'] = Dates['Winter'].apply(lambda x: 'Summer' if x == '-' else 'Winter')
del Dates['Winter']
Dates.columns = ['date','year','season']
Dates.insert(loc=0, column='date_id', value=np.nan)
for i in range(len(Dates)):
    Dates['date_id'][i] = str(str(Dates.year[i]) +' '+ Dates.season[i])
Dates.insert(loc=3, column='month', value=df_cities['Opening Ceremony'].apply(lambda x: int(datetime.datetime.strptime(x,'%Y-%m-%d').strftime('%m'))))
Dates.insert(loc=4, column='day', value=df_cities['Opening Ceremony'].apply(lambda x: int(datetime.datetime.strptime(x,'%Y-%m-%d').strftime('%d'))))

Dates

,date_id,date,year,month,day,season
0,2000 Summer,2000-09-15,2000,9,15,Summer
1,2002 Winter,2002-02-08,2002,2,8,Winter
2,2004 Summer,2004-08-13,2004,8,13,Summer
3,2006 Winter,2006-02-10,2006,2,10,Winter
4,2008 Summer,2008-08-08,2008,8,8,Summer
5,2010 Winter,2010-02-12,2010,2,12,Winter
6,2012 Summer,2012-07-27,2012,7,27,Summer
7,2014 Winter,2014-02-07,2014,2,7,Winter


## Utworzenie tabelki `Olympic games`

In [676]:
df_games_data.head()

,games,costs,athletes,events
0,1896 Summer,NaN,241,43
1,1900 Summer,NaN,997,95
2,1904 Summer,NaN,651,95
3,1906 Summer,NaN,854,78
4,1908 Summer,0.012,2008,110


In [677]:
Olympic_games = copy.deepcopy(df_games_data)
Olympic_games.columns = ['game_id', 'cost', 'athlete_number', 'events_number']
mean_cost = np.mean(Olympic_games.cost)
Olympic_games.cost.fillna(mean_cost, inplace=True)
Olympic_games.insert(loc=1, column='date_id', value=copy.deepcopy(Olympic_games.game_id))

Olympic_games.head()

,game_id,date_id,cost,athlete_number,events_number
0,1896 Summer,1896 Summer,3.664839,241,43
1,1900 Summer,1900 Summer,3.664839,997,95
2,1904 Summer,1904 Summer,3.664839,651,95
3,1906 Summer,1906 Summer,3.664839,854,78
4,1908 Summer,1908 Summer,0.012000,2008,110


Komentarze:
- ograniczenie tabelek ze względu na rok z kolumną postaci '1900 Summer'? (ograniczenie tabel wymiarowych)

## Utworzenie tabelki `Olympic event fact`

In [678]:
df_athlete_events.head(2)

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,birth_year,medal_score
0,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,lack,1989.0,0
1,9,Antti Sami Aalto,M,26.0,186.0,96.0,Finland,FIN,2002 Winter,2002,Winter,Salt Lake City,Ice Hockey,Ice Hockey Men's Ice Hockey,lack,1976.0,0


In [679]:
Olympic_event_fact = copy.deepcopy(df_athlete_events)[['Medal','ID','NOC','Games','Games','City','Event','medal_score']]
Olympic_event_fact.columns = ['medal_name','athlete_id','NOC','game_id','date_id','City','event_name','medal_score']
Olympic_event_fact.insert(loc=0, column='medal_id', value=range(len(Olympic_event_fact)))

Olympic_event_fact.head(3)

,medal_id,medal_name,athlete_id,NOC,game_id,date_id,City,event_name,medal_score
0,0,lack,2,CHN,2012 Summer,2012 Summer,London,Judo Men's Extra-Lightweight,0
1,1,lack,9,FIN,2002 Winter,2002 Winter,Salt Lake City,Ice Hockey Men's Ice Hockey,0
2,2,lack,12,FIN,2000 Summer,2000 Summer,Sydney,Badminton Men's Singles,0


# Uaktualnianie tabelek

In [680]:
Olympic_games.head()

,game_id,date_id,cost,athlete_number,events_number
0,1896 Summer,1896 Summer,3.664839,241,43
1,1900 Summer,1900 Summer,3.664839,997,95
2,1904 Summer,1904 Summer,3.664839,651,95
3,1906 Summer,1906 Summer,3.664839,854,78
4,1908 Summer,1908 Summer,0.012000,2008,110


In [681]:
Teams['NOC'].unique()

array(['CHN', 'FIN', 'NOR', 'NED', 'FRA', 'EST', 'MAR', 'EGY', 'ESP',
       'RUS', 'ARG', 'CUB', 'BLR', 'GRE', 'CMR', 'MEX', 'USA', 'ITA',
       'NCA', 'ALG', 'IRQ', 'PAK', 'QAT', 'IRI', 'AZE', 'CAN', 'AUS',
       'RSA', 'ERI', 'SUD', 'PLE', 'KUW', 'MAS', 'BRU', 'BRN', 'INA',
       'UZB', 'UAE', 'KGZ', 'TJK', 'JPN', 'GER', 'ETH', 'ISV', 'TUR',
       'SRI', 'CIV', 'KAZ', 'NGR', 'UKR', 'LAT', 'SYR', 'MLI', 'SWE',
       'GEO', 'CRO', 'POR', 'SUI', 'POL', 'BRA', 'ANG', 'COL', 'BAN',
       'JOR', 'LBA', 'SOM', 'ROU', 'VEN', 'IND', 'GBR', 'LIB', 'UGA',
       'TUN', 'SLO', 'MRI', 'NIG', 'SKN', 'NZL', 'GHA', 'LBR', 'SUR',
       'NEP', 'ARM', 'PLW', 'LTU', 'ASA', 'BEL', 'TOG', 'RWA', 'DMA',
       'DEN', 'MLT', 'AUT', 'SEY', 'DOM', 'BIZ', 'TTO', 'PUR', 'PAR',
       'URU', 'COM', 'MDV', 'BEN', 'NAM', 'PER', 'BER', 'SCG', 'HUN',
       'VIN', 'SAM', 'CYP', 'KEN', 'YEM', 'KSA', 'OMA', 'FIJ', 'VAN',
       'JAM', 'BUL', 'GUA', 'SRB', 'MON', 'IVB', 'ISL', 'ESA', 'CAF',
       'MAD', 'DJI',

In [682]:
test_df = Olympic_event_fact.join(df_noc_regions.set_index('NOC'), on='NOC', lsuffix='_left', rsuffix='_right', how='inner')#[['team_id','team_country','Team','NOC_right']]

In [683]:
df_noc_regions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231 entries, 0 to 230
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   NOC     231 non-null    object
 1   region  228 non-null    object
 2   notes   22 non-null     object
dtypes: object(3)
memory usage: 5.5+ KB


In [684]:
nocs = test_df['NOC'].unique()

In [685]:
df_noc_regions = df_noc_regions[df_noc_regions['NOC'].isin(nocs)]

In [686]:
df_noc_regions = df_noc_regions.fillna('Tuvalu')

In [687]:
df_noc_regions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 207 entries, 0 to 230
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   NOC     207 non-null    object
 1   region  207 non-null    object
 2   notes   207 non-null    object
dtypes: object(3)
memory usage: 6.5+ KB


In [688]:
test_df1 = df_noc_regions.join(Teams.set_index('team_country'), on='region', lsuffix='_left', rsuffix='_right', how='left')#[['team_id','team_country','Team','NOC_right']]

In [689]:
diff = test_df1[test_df1['team_name'].isnull()]

In [690]:
Teams = test_df1[['NOC_left','team_id','region', 'region']]

In [691]:
Teams

,NOC_left,team_id,region,region
0,AFG,192.0,Afghanistan,Afghanistan
1,AHO,NaN,Curacao,Curacao
2,ALB,211.0,Albania,Albania
3,ALG,19.0,Algeria,Algeria
4,AND,168.0,Andorra,Andorra
...,...,...,...,...
221,VIN,NaN,Saint Vincent,Saint Vincent
225,YEM,121.0,Yemen,Yemen
228,ZAM,239.0,Zambia,Zambia
229,ZIM,183.0,Zimbabwe,Zimbabwe


In [692]:
Teams = Teams.rename(columns = {'NOC_left':'NOC','region':'team_country'})

In [693]:
Teams = Teams.iloc[:,0:3] #team country to ta sama kolumna

In [694]:
Teams['team_id'] = range(0,len(Teams))

In [695]:
Olympic_games = Dates.join(Olympic_games.set_index('game_id'), how='inner', lsuffix='_left', rsuffix='_right', on='date_id')[['date_id','cost','athlete_number','events_number']]

In [696]:
Olympic_games = Olympic_games.rename(columns={'date_id':'game_id'})

In [697]:
Olympic_games['date_id'] = range(0,len(Olympic_games))

In [698]:
Dates['date_id'] = range(0,len(Dates))

In [699]:
new_row = {'date_id':8,'date':date.max,'year':9999,'month':12,'day':31,'season':"Summer"}

In [700]:
Dates = Dates.append(new_row,ignore_index=True)

In [701]:
import random

In [702]:
Athletes['date_valid_from'] = random.choices(range(0,8),k=len(Athletes))

In [703]:
Athletes['date_valid_to'] = [8 for i in range(len(Athletes))]

In [704]:
Sports.head()

,event_id,sport_name,event_name
0,0,Judo,Judo Men's Extra-Lightweight
1,1,Ice Hockey,Ice Hockey Men's Ice Hockey
2,2,Badminton,Badminton Men's Singles
3,3,Sailing,Sailing Women's Windsurfer
4,4,Athletics,Athletics Men's Shot Put


In [705]:
Sports['sport_type'] = ["Individual" if sport in(sports_types_dict['individual_sport']) else "Group" for sport in Sports['sport_name']]

# Podsumowanie tabelek

In [706]:
Olympic_games #gotowa

,game_id,cost,athlete_number,events_number,date_id
0,2000 Summer,5.026,10651,300,0
1,2002 Winter,2.520,2399,78,1
2,2004 Summer,2.942,10625,301,2
3,2006 Winter,4.366,2508,84,3
4,2008 Summer,6.810,10942,302,4
5,2010 Winter,2.540,2566,86,5
6,2012 Summer,14.957,10568,302,6
7,2014 Winter,21.890,2780,98,7


In [707]:
Dates #gotowe

,date_id,date,year,month,day,season
0,0,2000-09-15,2000,9,15,Summer
1,1,2002-02-08,2002,2,8,Winter
2,2,2004-08-13,2004,8,13,Summer
3,3,2006-02-10,2006,2,10,Winter
4,4,2008-08-08,2008,8,8,Summer
5,5,2010-02-12,2010,2,12,Winter
6,6,2012-07-27,2012,7,27,Summer
7,7,2014-02-07,2014,2,7,Winter
8,8,9999-12-31,9999,12,31,Summer


In [708]:
Athletes.head() #gotowa

,sid,athlete_id,full_name,sex,birth_year,height,weight,score,date_valid_from,date_valid_to,status
0,0,2,A Lamusi,M,1989.0,170.0,60.0,4,5,8,1
1,1,9,Antti Sami Aalto,M,1976.0,186.0,96.0,4,7,8,1
2,2,12,Jyri Tapani Aalto,M,1969.0,172.0,70.0,1,6,8,1
3,3,13,Minna Maarit Aalto,F,1966.0,159.0,55.5,5,3,8,1
4,4,16,Juhamatti Tapio Aaltonen,M,1986.0,184.0,85.0,1,4,8,1


In [709]:
Locations #gotowa

,location_id,continent,country,city
0,0,Oceania,Australia,Sydney
1,1,North America,United States,Salt Lake City
2,2,Europe,Greece,Athens
3,3,Europe,Italy,Turin
4,4,Asia,China,Beijing
5,5,North America,Canada,Vancouver
6,6,Europe,United Kingdom,London
7,7,Europe,Russia,Sochi


In [710]:
Olympic_event_fact['City'].unique()

array(['London', 'Salt Lake City', 'Sydney', 'Sochi', 'Torino', 'Beijing',
       'Athina', 'Vancouver'], dtype=object)

In [711]:
Locations['city'] = ['Sydney', 'Salt Lake City', 'Athina', 'Torino', 'Beijing', 'Vancouver', 'London', 'Sochi']

In [712]:
Teams#gotowa

,NOC,team_id,team_country
0,AFG,0,Afghanistan
1,AHO,1,Curacao
2,ALB,2,Albania
3,ALG,3,Algeria
4,AND,4,Andorra
...,...,...,...
221,VIN,202,Saint Vincent
225,YEM,203,Yemen
228,ZAM,204,Zambia
229,ZIM,205,Zimbabwe


In [713]:
Sports #gotowa

,event_id,sport_name,event_name,sport_type
0,0,Judo,Judo Men's Extra-Lightweight,Individual
1,1,Ice Hockey,Ice Hockey Men's Ice Hockey,Group
2,2,Badminton,Badminton Men's Singles,Individual
3,3,Sailing,Sailing Women's Windsurfer,Group
4,4,Athletics,Athletics Men's Shot Put,Individual
...,...,...,...,...
430,430,Trampolining,Trampolining Men's Individual,Group
431,431,Weightlifting,Weightlifting Women's Flyweight,Group
432,432,Cycling,Cycling Women's Omnium,Individual
433,433,Cycling,Cycling Women's Team Sprint,Individual


In [714]:
Olympic_event_fact

,medal_id,medal_name,athlete_id,NOC,game_id,date_id,City,event_name,medal_score
0,0,lack,2,CHN,2012 Summer,2012 Summer,London,Judo Men's Extra-Lightweight,0
1,1,lack,9,FIN,2002 Winter,2002 Winter,Salt Lake City,Ice Hockey Men's Ice Hockey,0
2,2,lack,12,FIN,2000 Summer,2000 Summer,Sydney,Badminton Men's Singles,0
3,3,lack,13,FIN,2000 Summer,2000 Summer,Sydney,Sailing Women's Windsurfer,0
4,4,Bronze,16,FIN,2014 Winter,2014 Winter,Sochi,Ice Hockey Men's Ice Hockey,1
...,...,...,...,...,...,...,...,...,...
71565,71565,lack,135567,RUS,2000 Summer,2000 Summer,Sydney,Rowing Men's Lightweight Coxless Fours,0
71566,71566,lack,135567,RUS,2004 Summer,2004 Summer,Athina,Rowing Men's Lightweight Coxless Fours,0
71567,71567,lack,135570,POL,2014 Winter,2014 Winter,Sochi,"Ski Jumping Men's Large Hill, Individual",0
71568,71568,lack,135570,POL,2014 Winter,2014 Winter,Sochi,"Ski Jumping Men's Large Hill, Team",0


In [715]:
dict_date = {key: value for (key,value) in zip(Olympic_games['game_id'], Olympic_games['date_id'])}

In [716]:
Olympic_event_fact['date_id'] = Olympic_event_fact['date_id'].map(dict_date)

In [717]:
dict_reg = {key: value for (key,value) in zip(Teams['NOC'], Teams['team_id'])}

In [718]:
Olympic_event_fact['team_id'] = Olympic_event_fact['NOC'].map(dict_reg)

In [719]:
dict_loc = {key: value for (key,value) in zip(Locations['city'], Locations['location_id'])}

In [720]:
Olympic_event_fact['location_id'] = Olympic_event_fact['City'].map(dict_loc)

In [721]:
dict_ev = {key: value for (key,value) in zip(Sports['event_name'], Sports['event_id'])}

In [722]:
Olympic_event_fact['event_id'] = Olympic_event_fact['event_name'].map(dict_ev)

In [723]:
Olympic_event_fact.drop(['event_name', 'NOC', 'City'],axis=1, inplace=True)

In [724]:
Olympic_event_fact.head()

,medal_id,medal_name,athlete_id,game_id,date_id,medal_score,team_id,location_id,event_id
0,0,lack,2,2012 Summer,6,0,39,6,0
1,1,lack,9,2002 Winter,1,0,64,1,1
2,2,lack,12,2000 Summer,0,0,64,0,2
3,3,lack,13,2000 Summer,0,0,64,0,3
4,4,Bronze,16,2014 Winter,7,1,64,7,1
